# Testing Battery Decision Reasoning with Heuristic Solver

This notebook demonstrates the reasoning pipeline integrated with the heuristic solver to explain battery charging/discharging decisions.

**What this shows:**
- Running heuristic optimization (time-based and quantile-based strategies)
- Using LLM to generate natural language explanations for battery decisions
- Visualizing decisions with their context

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Import project modules
# Note: Importing agentic_energy.agentics_reasoning automatically applies the Agentics framework patch
from agentic_energy.heuristics.heuristic_trader import HeuristicTrader, records_to_arrays
from agentic_energy.data_loader import EnergyDataLoader
from agentic_energy import (
    BatteryParams, DayInputs, SolveRequest, SolveResponse,
    ReasoningRequest, ReasoningResponse, BatteryReasoningAG
)

print("✓ Imports successful (Agentics patch auto-applied)")

## Load Test Data
Load Italy market data for testing

In [ ]:
# Load Italy market data
italy_obj = EnergyDataLoader(region="ITALY")
italy_data = italy_obj.load_region_data()

# Get first day of data (24 hours)
day_records = italy_data[0:24]
prices, demand = records_to_arrays(day_records)

print(f"Loaded {len(day_records)} hours of data")
print(f"Price range: ${prices.min():.2f} - ${prices.max():.2f}/kWh")
print(f"Demand range: {demand.min():.2f} - {demand.max():.2f} kW")

## Setup Battery and Heuristic Strategies

In [ ]:
# Define battery parameters
battery = BatteryParams(
    capacity_kwh=49.44,
    cmax_kw=12.36,
    dmax_kw=12.36,
    soc_init=0.5,
    soc_min=0.0,
    soc_max=1.0,
    eta_c=0.95,
    eta_d=0.95,
    soc_target=0.5,
)

# Create day inputs
day = DayInputs(
    prices_buy=prices.tolist(),
    prices_sell=prices.tolist(),
    demand_kw=demand.tolist(),
    allow_export=True,
    dt_hours=1.0,
)

# Create time-based heuristic trader
trader_time = HeuristicTrader(
    mode="time",
    charge_windows=[(2, 6), (10, 16), (20, 22)],
    discharge_windows=[(0, 2), (6, 10), (16, 20), (22, 24)],
)

# Create solve request
req_time = SolveRequest(
    battery=battery,
    day=day,
    solver=None,
    solver_opts={"mode": "time"},
)

print("✓ Battery and strategy configured")

## Run Heuristic Optimization

In [ ]:
# Run the time-based heuristic
result_time = trader_time.solve(req_time)

print(f"Time-based strategy results:")
print(f"  Status: {result_time.status}")
print(f"  Objective cost: ${result_time.objective_cost:.2f}")
print(f"  Decisions: {len(result_time.decision)} timesteps")

# Summary of decisions
decisions_array = np.array(result_time.decision)
n_charge = np.sum(decisions_array == 1.0)
n_discharge = np.sum(decisions_array == -1.0)
n_idle = np.sum(decisions_array == 0.0)
print(f"  Charge: {n_charge} hours, Discharge: {n_discharge} hours, Idle: {n_idle} hours")

## Initialize Reasoning System

Create the LLM-based reasoning system that will explain the decisions.

In [ ]:
# Initialize reasoning system with Gemini
# You can also try "openai" or other providers
reasoning = BatteryReasoningAG(llm_provider="gemini")

print("✓ Reasoning system initialized")

## Generate Explanations for Key Decisions

Let's explain decisions at a few interesting timesteps (beginning, middle, end of day)

In [ ]:
# Pick interesting timesteps to explain
interesting_hours = [0, 6, 12, 18]  # Midnight, morning, noon, evening

print("Generating explanations for selected hours...\n")
print("="*80)

# Generate explanations
explanations = await reasoning.explain_sequence(
    solve_request=req_time,
    solve_response=result_time,
    indices=interesting_hours
)

# Display explanations
decision_labels = {1.0: "CHARGE", 0.0: "IDLE", -1.0: "DISCHARGE"}

for hour, explanation in zip(interesting_hours, explanations):
    decision = result_time.decision[hour]
    soc = result_time.soc[hour]
    price = prices[hour]
    
    print(f"\n{'='*80}")
    print(f"HOUR {hour} - Decision: {decision_labels.get(decision, decision)}")
    print(f"SoC: {soc:.1%} | Price: ${price:.2f}/kWh | Demand: {demand[hour]:.2f} kW")
    print(f"{'-'*80}")
    print(f"\nEXPLANATION (Confidence: {explanation.confidence:.0%}):\n")
    print(explanation.explanation)
    
    if explanation.key_factors:
        print(f"\nKEY FACTORS:")
        for factor in explanation.key_factors:
            print(f"  • {factor}")
    
    if explanation.supporting_data:
        print(f"\nSUPPORTING DATA:")
        for key, value in explanation.supporting_data.items():
            print(f"  • {key}: {value:.4f}")

print(f"\n{'='*80}")

## Visualize Results

Plot the decisions alongside prices and SoC to see the patterns

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

hours = np.arange(24)

# Plot 1: Prices with explained timesteps marked
axes[0].plot(hours, prices, 'k-', linewidth=2, label='Price')
axes[0].scatter(interesting_hours, [prices[i] for i in interesting_hours], 
                color='red', s=100, zorder=5, label='Explained timesteps')
axes[0].set_ylabel('Price ($/kWh)', fontsize=12)
axes[0].set_title('Energy Prices Throughout the Day', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: State of Charge
axes[1].plot(hours, result_time.soc, 'b-', linewidth=2, label='SoC')
axes[1].scatter(interesting_hours, [result_time.soc[i] for i in interesting_hours],
                color='red', s=100, zorder=5)
axes[1].axhline(y=battery.soc_min, color='r', linestyle='--', alpha=0.5, label='Min SoC')
axes[1].axhline(y=battery.soc_max, color='g', linestyle='--', alpha=0.5, label='Max SoC')
axes[1].set_ylabel('State of Charge', fontsize=12)
axes[1].set_title('Battery State of Charge Over Time', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 1])

# Plot 3: Decisions
colors = ['red' if d == -1 else 'green' if d == 1 else 'gray' for d in result_time.decision]
axes[2].bar(hours, result_time.decision, color=colors, alpha=0.6, edgecolor='black')
axes[2].scatter(interesting_hours, [result_time.decision[i] for i in interesting_hours],
                color='darkred', s=100, zorder=5, marker='*', label='Explained')
axes[2].set_ylabel('Decision', fontsize=12)
axes[2].set_xlabel('Hour of Day', fontsize=12)
axes[2].set_title('Battery Decisions (Green=Charge, Red=Discharge, Gray=Idle)', fontsize=14, fontweight='bold')
axes[2].set_yticks([-1, 0, 1])
axes[2].set_yticklabels(['Discharge', 'Idle', 'Charge'])
axes[2].legend()
axes[2].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n✓ Visualization complete")

## Summary

This notebook demonstrates:

✅ **Reasoning Pipeline Integration**: LLM-based reasoning layer successfully explains battery decisions  
✅ **Heuristic Optimization**: Time-based strategy optimizes battery charging/discharging  
✅ **Natural Language Explanations**: Clear explanations of why decisions were made  
✅ **Confidence Scores**: Each explanation includes a confidence level  
✅ **Key Factors**: Highlights important factors influencing each decision  
✅ **Visualization**: Visual context for understanding decision patterns  

## Next Steps

- Try different heuristic strategies (quantile-based, etc.)
- Compare explanations across different optimization algorithms (MILP, RL, Heuristics)
- Adjust the reasoning instructions for different types of explanations
- Test on different market regions and time periods